# Draft: STIPS Tutorial

***

## Introduction

**STIPS**, or the Space Telescope Imaging Product Simulator, is a tool developed by the Space Telescope Science Institute for simulating observations with the full array of detectors on the Roman Wide Field Imager (WFI).

Though it trades some accuracy in order to capture the full array of detectors simulations – see the Pandeia Tutorial for higher accuracy simulations of smaller areas – STIPS simulations do take Roman's exposure-level pipeline ("Level 2") into account. This means generated scenes come with readouts of many calibration residuals. Scenes are also returned with Poisson and readout noise estimates and can incorporate instrumental distortion.

This notebook is a starter guide to simulating and manipulating scenes with STIPS. STIPS requires separate reference data both for itself and for some of its dependencies, so if you're running this notebook locally, be sure you've followed [the complete installation instructions](https://stips.readthedocs.io/en/latest/installation.html) before attempting to run it.

## Imports

Besides the STIPS-related imports, the `matplotlib` imports will help visualize simulated images and `astropy.io.fits` will help write a FITS table on the fly.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import stips

from astropy.io import fits
from stips.astro_image import AstroImage
from stips.observation_module import ObservationModule
from stips.scene_module import SceneModule

### Environment report

To verify the existing STIPS installation alongside its associated data files and dependencies, run the cell below. (Find the current software requirements [in the STIPS documentation](https://stips.readthedocs.io/en/latest/installation.html#stips-requirements).)

In [ ]:
print(stips.__env__report__)

***

## Examples

### Basic STIPS usage

At its most fundamental level, STIPS takes a dictionary of observation and instrument parameters and a FITS source catalog in order to return a simulated image.

In this example, we build a dictionary specifying that the image is taken with the F129 filter, comes from the WFI01 detector, and has an exposure time of 300 seconds.

In [ ]:
obs = {'instrument': 'WFI', 'filters': ['F129'], 'detectors': 1,
       'background': 'pandeia', 'observations_id': 42, 'exptime': 300,
       'offsets': [{'offset_id': 1, 'offset_centre': False, 'offset_ra': 0.0,
                    'offset_dec': 0.0, 'offset_pa': 0.0}]
       }

Feed the dictionary to an instance of the `ObservationModule` class, while also specifying the central coordinates (90 degrees right ascension and 30 degrees declination) and position angle (0 degrees) as keyword arguments.

In [ ]:
obm = ObservationModule(obs, ra=90, dec=30, pa=0, seed=42, cores=6)

#### Create a simple astronomical scene

The other requirement is the FITS source catalog. STIPS accepts [several types of tables and catalogs](https://stips.readthedocs.io/en/latest/using_stips/catalogue_formats.html#stips-table-formats) in either the IPAC text or FITS BinTable format. This example uses a Mixed Catalog, which requires the following columns:

* `id`: Object ID
* `ra`: Right ascension (RA), in degrees
* `dec`: Declination (DEC), in degrees
* `flux`: Flux, in `units` (defined below)
* `type`: Approximation used to profile a source. Options are `'sersic'` (for extended sources) or `point`
* `n`: Sersic profile index
* `phi`: Position angle of the Sersic profle's major axis, in degrees
* `ratio`: Ratio of the Sersic profile's major and minor axes
  * Since `n`, `phi`, `ratio` only apply to extended sources, they are ignored in rows where `type` is set to `'point'`.
* `notes`: Optional per-source comments
* `units`: One of `‘p’` for photons/s, `‘e’` for electrons/s, `‘j’` for Jansky, or `‘c’` for counts/s

The catalog we create below contains two sources located near the central coordinates specified in the `ObservationModule`.

In [ ]:
cols = [
    fits.Column(name='id', array=[1, 2], format='K'),
    fits.Column(name='ra', array=[90.02, 90.03], format='E'),
    fits.Column(name='dec', array=[29.98, 29.97], format='D'),
    fits.Column(name='flux', array=[0.00023, 0.0004], format='D'),
    fits.Column(name='type', array=['point', 'point'], format='8A'),
    fits.Column(name='n', array=[0, 0], format='D'),
    fits.Column(name='re', array=[0, 0], format='D'),
    fits.Column(name='phi', array=[0, 0], format='D'),
    fits.Column(name='ratio', array=[0, 0], format='D'),
    fits.Column(name='notes', array=['', ''], format='8A'),
    fits.Column(name='units', array=['j', 'j'], format='8A')
]

Next, use the columns created above to create an output FITS table in the BinTable format and assign header keys that specify the filter and catalog type mentioned earlier.

In [ ]:
hdut = fits.BinTableHDU.from_columns(cols)
hdut.header['TYPE'] = 'mixed'
hdut.header['FILTER'] = 'F129'

From there, save the table to disk.

In [ ]:
cat_file = 'catalog.fits'
hdut.writeto(cat_file, overwrite=True)

#### Simulate an image

With the observation module and source catalog in tow, STIPS can take over the image simulation process. First, trigger the initialization of a new observation.

In [ ]:
obm.nextObservation()

Next, simulate an image containing sources from the catalog you created earlier. Then, and add error residuals to the image.

In [ ]:
cat_name = obm.addCatalogue(cat_file)

In [ ]:
obm.addError()

Finish by saving the outputs to a FITS file. `ObservationModule`'s `finalize()` method can also return a mosaic and a list of the simulation's initial parameters.

In [ ]:
fits_file_1, _, params_1 = obm.finalize(mosaic=False)
print(f"Output FITS file is {fits_file_1}")

In [ ]:
for prm in params_1:
    print(prm)

In [ ]:
img_1 = fits.getdata(fits_file_1)[1000:1500, 2500:3000]

plt.imshow(img_1, vmax=1.5, origin='lower', cmap='bone')

### Generate scenes from user-created catalogs

STIPS can simulate scenes by importing pre-existing catalogs (as in the first example) or by using built-in functionality that generates collections of stars or galaxies based on user-specified parameters. Below, we will use the STIPS `SceneModule` to generate a stellar population and a galactic population.

#### Create a stellar population

First, we specify parameters that will be the same in both scenes and use them to initalize separate `SceneModule` instances.

In [ ]:
obs_prefix_1 = 'notebook_example1'
obs_ra = 150.0
obs_dec = -2.5

scm_stellar = SceneModule(out_prefix=obs_prefix_1, ra=obs_ra, dec=obs_dec)
scm_galactic = SceneModule(out_prefix=obs_prefix_1, ra=obs_ra, dec=obs_dec)

Next, create a dictionary containing parameters of the desired stellar population to pass to one of the `SceneModule` instances' `CreatePopulation()` method.

In [ ]:
stellar_parameters = {'n_stars': 100, 'age_low': 7.5e12, 'age_high': 7.5e12,
                      'z_low': -2.0, 'z_high': -2.0, 'imf': 'salpeter',
                      'alpha': -2.35, 'binary_fraction': 0.1,
                      'distribution': 'invpow', 'clustered': True,
                      'radius': 100.0, 'radius_units': 'pc',
                      'distance_low': 20.0, 'distance_high': 20.0,
                      'offset_ra': 0.0, 'offset_dec': 0.0
                      }

_(Find a full accounting of each dictionary entry's meaning in the `CreatePopulation()` docstring.)_

In [ ]:
scm_stellar.CreatePopulation?

Pass the dictionary to the proper `SceneModule` instance's `CreatePopulation()` method. Running the method will save the newly-generated population to disk.

In [ ]:
stellar_cat_file = scm_stellar.CreatePopulation(stellar_parameters)
print(f"Stellar population saved to file {stellar_cat_file}")

#### Create a galactic population

Repeat the population generation process, now by creating a dictionary containing parameters of a desired _galactic_ population, passing it to the other `SceneModule` instance's `CreateGalaxies()` method, and saving that result to disk.

In [ ]:
galaxy_parameters = {'n_gals': 10, 'z_low': 0.0, 'z_high': 0.2,
                     'rad_low': 0.01, 'rad_high': 2.0,
                     'sb_v_low': 30.0, 'sb_v_high': 25.0,
                     'distribution': 'uniform', 'clustered': False,
                     'radius': 200.0, 'radius_units': 'arcsec',
                     'offset_ra': 0.0, 'offset_dec': 0.0
                     }

In [ ]:
scm_galactic.CreateGalaxies?

In [ ]:
galaxy_cat_file = scm_galactic.CreateGalaxies(galaxy_parameters)
print(f"Galaxy population saved to file {galaxy_cat_file}")

#### Set up an observation (first pointing)

Once you've created a scene, you can use STIPS to simulate as many exposures of it in as many orientations as you'd like. In STIPS, a single telescope pointing is called an _offset_, and a collection of exposures is an _observation_.

We start this subsection by creating a single _offset_ that is dithered by 2 degrees in right ascension and rotated by 0.5 degrees in position angle from the center of the scene.

In [ ]:
offset_1 = {'offset_id': 1, 'offset_centre': False,
            # True would center each detector on the same on-sky point
            'offset_ra': 2.0, 'offset_dec': 0.0, 'offset_pa': 0.5
            }

That offset is contained within an _observation_ that is taken with the F129 filter, uses detectors WFI01 through WFI03, and has an exposure time of 1500 seconds. We also apply distortion and specify a sky background of 0.24 counts/s/pixel.

In [ ]:
observation_parameters_1 = {'instrument': 'WFI', 'filters': ['F129'],
                            'detectors': 3, 'distortion': True,
                            'background': 0.24, 'observations_id': 1,
                            'exptime': 1500, 'offsets': [offset_1]
                            }

STIPS can also apply various types of error residuals to the observation. Here, we only include residuals from flatfielding and the dark current.

In [ ]:
residuals_1 = {'residual_flat': True, 'residual_dark': True,
               'residual_cosmic': False, 'residual_poisson': False,
               'residual_readnoise': False
               }

Next, feed the dictionaries of observation and residual parameters to an instance of the `ObservationModule` class, alongside the observation prefix, right ascension, and declination specified earlier in this example.

In [ ]:
obm_1 = ObservationModule(observation_parameters_1, residual=residuals_1,
                          out_prefix=obs_prefix_1, ra=obs_ra, dec=obs_dec)

Call the `ObservationModule` object's `nextObservation()` method to move to the first offset/filter combination (`offset_1` and F129) contained in the object.

In [ ]:
obm_1.nextObservation()

Now that the observation is ready, add the stellar and galactic populations to it. (Each population may take about a minute to load.) Follow up by adding in the sources of error specified upon its initialization.

In [ ]:
output_stellar_catalogs_1 = obm_1.addCatalogue(stellar_cat_file)
output_galaxy_catalogs_1 = obm_1.addCatalogue(galaxy_cat_file)
print(f"Output Catalogs are {output_stellar_catalogs_1} and "
      f"{output_galaxy_catalogs_1}.")

In [ ]:
obm_1.addError()

As before, finish by saving the simulated image to a FITS file.

In [ ]:
fits_file_1, _, params_1 = obm_1.finalize(mosaic=False)
print(f"Output FITS file is {fits_file_1}")

In [ ]:
img_1 = fits.getdata(fits_file_1, ext=1)

norm_1 = mpl.colors.LogNorm(vmin=img_1.min(), vmax=img_1.max())
plt.imshow(img_1, norm=norm_1, origin='lower', cmap='bone')

#### Modify an observation (second pointing)

To observe the same scene under different conditions, make a new `ObservationModule` object that takes updated versions of the input observation parameter and residual dictionaries. Taken collectively, the collection of resulting `ObservationModule` objects can be thought of as a dithered set of observations.

For the second observation, create another offset that is dithered by 10 degrees in right ascension and rotated by 27 degrees in position angle from the center of the scene.

In [ ]:
offset_2 = {'offset_id': 1, 'offset_centre': False,
            # True centers each detector on same point
            'offset_ra': 10.0, 'offset_dec': 0.0, 'offset_pa': 27
            }

The second observation is the same as the first (F129 filter, detectors WFI01 through WFI03, exposure time of 1500 seconds, distortion, and a sky background of 0.24 counts/s/pixel) besides the substitution of the first set of offset parameters for the second.

In [ ]:
observation_parameters_2 = {'instrument': 'WFI', 'filters': ['F129'],
                            'detectors': 3, 'distortion': True,
                            'background': 0.24, 'observations_id': 1,
                            'exptime': 1500, 'offsets': [offset_2]
                            }

This time, we include residuals from the flatfield (like before) and read noise (unlike before).

In [ ]:
residuals_2 = {'residual_flat': True, 'residual_dark': False,
               'residual_cosmic': False, 'residual_poisson': False,
               'residual_readnoise': True
               }

Create the new `ObservationModule` object and initialize it for a simulation.

In [ ]:
obs_prefix_2 = 'notebook_example2'
obm_2 = ObservationModule(observation_parameters_2, residuals=residuals_2,
                          out_prefix=obs_prefix_2, ra=obs_ra, dec=obs_dec)

In [ ]:
obm_2.nextObservation()

Add the stellar and galactic populations to the new observation along with the sources of error chosen above.

In [ ]:
output_stellar_catalogs_2 = obm_2.addCatalogue(stellar_cat_file)
output_galaxy_catalogs_2 = obm_2.addCatalogue(galaxy_cat_file)
print(f"Output Catalogs are {output_stellar_catalogs_2} and "
      f"{output_galaxy_catalogs_2}.")

In [ ]:
obm_2.addError()

In [ ]:
fits_file_2, _, params_2 = obm_2.finalize(mosaic=False)
print(f"Output FITS file is {fits_file_2}")

Below, see the two pointings side by side.

In [ ]:
img_2 = fits.getdata(fits_file_2, ext=1)
norm_2 = mpl.colors.LogNorm(vmin=img_2.min(), vmax=img_2.max())

fig_2, ax_2 = plt.subplots(1, 2, figsize=(8, 4))
ax_2[0].imshow(img_1, norm=norm_1, origin='lower', cmap='bone')
ax_2[1].imshow(img_2, norm=norm_2, origin='lower', cmap='bone')

### Add artifical point source to an observation

With the STIPS `makePSF` utility, users can "clip" a PSF from a given detector pixel position in a scene and inject it elsewhere.

This is achieved by 1) applying a bi-linear interpolation of a 3x3 array from the STIPS PSF library to compute the best PSF at the specified integer SCA pixels, then 2) performing bicubic interpolations over the PSF's supersampled pixel grid to fill out its sub-pixel positions. The resulting PSF can then be injected elsewhere in an existing scene or used to create new scenes.

Use the `make_epsf_array()` method from STIPS' `AstroImage` class to create an example PSF from the library.

In [ ]:
ai = AstroImage()

ai.detector = 'SCA01'
ai.filter = 'F129'
test_psf = ai.make_epsf_array()[0][0][0]

In [ ]:
plt.imshow(test_psf, cmap='bone')

Specify the pixel coordinates of the source's center (which are just the center of the PSF in this case) and the pixel length of the intended cutout (1/8th of the full example PSF's pixel length).

In [ ]:
psf_mid_pixel = (test_psf.shape[0] - 1) // 2
boxsize = test_psf.shape[0] // 8

Inject the source at pixel (2000, 2000) with a flux of 3000 count/s _**(??)**_ in a copy of the simulated image from the previous section's second (final) observation. Can you see the newly injected PSF in the comparison plot?

In [ ]:
img_3 = img_2.copy()
xpix = 2000
ypix = 2000
flux = 3000

img_3_inj = stips.utilities.makePSF.place_source(xpix, ypix, flux, img_3,
                                                 test_psf, boxsize=boxsize,
                                                 psf_center=psf_mid_pixel)

In [ ]:
norm_3 = mpl.colors.LogNorm(vmin=img_3_inj.min(), vmax=img_3_inj.max())

fig_3, ax_3 = plt.subplots(1, 2, figsize=(8, 4))
ax_3[0].imshow(img_2, norm=norm_2, origin='lower', cmap='bone')
ax_3[1].imshow(img_3_inj, norm=norm_3, origin='lower', cmap='bone')

for ax in ax_3:
    ax.add_patch(plt.Circle((xpix, ypix), 75, color='r', alpha=.7, fill=False))

## Aditional Resources

- The Roman User Documentation's ["STIPS Overview"](https://roman-docs.stsci.edu/simulation-tools-handbook-home/stips-space-telescope-imaging-product-simulator/overview-of-stips) page and associated tutorials.
- The latest version of the [STIPS documentation](https://stips.readthedocs.io/en/latest/) on ReadTheDocs.
- The [Roman Help Desk](https://roman-docs.stsci.edu/roman-help-desk-at-stsci), an official outlet for user questions about STIPS.

## About this notebook

**Author:** Justin Otor, Staff Scientist II.  
**Updated In:** 2024-05

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 